In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from collections import Counter

# Image Processing
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# Model Building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Data Balancing
from imblearn.over_sampling import RandomOverSampler

# Visualization Style
sns.set_style('darkgrid')
warnings.filterwarnings("ignore")

In [ ]:
# Load metadata
data = pd.read_csv('/kaggle/input/isic-2019-skin-lesion-images-for-classification/ISIC_2019_Training_GroundTruth.csv')
print("Data Shape:", data.shape)

# Check image folder structure
dataset_path = '/kaggle/input/isic-2019-skin-lesion-images-for-classification/'
print(os.listdir(dataset_path))

In [ ]:
def load_and_resize_images(image_paths, target_size=(32, 32)):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=target_size)  # Load and resize
        img_array = img_to_array(img)  # Convert to array
        images.append(img_array)
    return np.array(images)

In [ ]:
# Define classes
class_folders = ['MEL', 'VASC', 'SCC', 'DF', 'NV', 'BKL', 'BCC', 'AK']

# Load images and labels
all_images, all_labels = [], []

for class_name in class_folders:
    class_folder_path = os.path.join(dataset_path, class_name)
    image_files = os.listdir(class_folder_path)
    class_images = load_and_resize_images([os.path.join(class_folder_path, img) for img in image_files])
    all_images.append(class_images)
    all_labels.append([class_name] * len(image_files))

# Combine all data
X_all = np.concatenate(all_images, axis=0)
y_all = np.concatenate(all_labels, axis=0)

print("Shape of all images:", X_all.shape)
print("Shape of all labels:", y_all.shape)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_all)
y_one_hot = to_categorical(y_encoded)

# Reshape data for oversampling
X_reshaped = X_all.reshape(X_all.shape[0], -1)
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_reshaped, y_one_hot)

print("Class distribution after oversampling:", Counter(np.argmax(y_resampled, axis=1)))

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(f"Shapes - X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

In [ ]:
# Reshape to (28, 28, 3)
X_train_reshaped = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test_reshaped = X_test.reshape(X_test.shape[0], 32, 32, 3)

In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
]
# Define input shape
input_shape = (32, 32, 3)
num_classes = 8  # Adjust based on your dataset

# Load Pretrained Models

def build_resnet50():
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Train and Evaluate Models
def train_and_evaluate(model, name):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test),
                        epochs=10, batch_size=64, callbacks=callbacks)
    test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
    print(f"{name} Test Accuracy: {test_accuracy * 100:.2f}%")
    return history, test_accuracy

# Train models
models = {
    "ResNet50": build_resnet50(),
}

histories, accuracies = {}, {}
for name, model in models.items():
    histories[name], accuracies[name] = train_and_evaluate(model, name)

# Plot Comparison
plt.figure(figsize=(10, 5))
plt.bar(accuracies.keys(), accuracies.values(), color=['red', 'blue', 'green', 'purple'])
plt.xlabel("Model")
plt.ylabel("Test Accuracy")
plt.title("Comparison of CNN Models")
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define callback
callbacks = [EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)]

# Define input shape & number of classes
input_shape = (32, 32, 3)
num_classes = 8  # Adjust based on your dataset

# Build ResNet50 Model
def build_resnet50():
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Train & Save Model
def train_and_save(model, model_name="ResNet50"):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test),
                        epochs=10, batch_size=64, callbacks=callbacks)
    
    test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
    print(f"{model_name} Test Accuracy: {test_accuracy * 100:.2f}%")

    # Save model
    model.save(f"{model_name}.h5")

    # Save training history and accuracy in JSON
    history_dict = history.history
    history_dict["test_accuracy"] = test_accuracy  # Store test accuracy separately

    with open(f"{model_name}_history.json", "w") as f:
        json.dump(history_dict, f)

    return history, test_accuracy

# Train and Save ResNet50
model = build_resnet50()
history, accuracy = train_and_save(model)

# Save Accuracy Separately
accuracy_data = {"ResNet50": accuracy}
with open("model_accuracies.json", "w") as f:
    json.dump(accuracy_data, f)

# Plot & Save Accuracy Graph
plt.figure(figsize=(6, 4))
plt.bar(["ResNet50"], [accuracy * 100], color="blue")
plt.xlabel("Model")
plt.ylabel("Test Accuracy (%)")
plt.title("ResNet50 Accuracy")
plt.savefig("resnet50_accuracy.png")  # Save plot as an image file
plt.show()


inceptionNET-----------------------

In [ ]:
def load_and_resize_images(image_paths, target_size=(75, 75)):
    images = []
    for path in image_paths:
        img = load_img(path, target_size=target_size)  # Load and resize
        img_array = img_to_array(img)  # Convert to array
        images.append(img_array)
    return np.array(images)

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from collections import Counter

# Image Processing
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

# Model Building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define classes
class_folders = ['MEL', 'VASC', 'SCC', 'DF', 'NV', 'BKL', 'BCC', 'AK']

# Load images and labels
all_images, all_labels = [], []

for class_name in class_folders:
    class_folder_path = os.path.join(dataset_path, class_name)
    image_files = os.listdir(class_folder_path)
    class_images = load_and_resize_images([os.path.join(class_folder_path, img) for img in image_files])
    all_images.append(class_images)
    all_labels.append([class_name] * len(image_files))

# Combine all data
X_all = np.concatenate(all_images, axis=0)
y_all = np.concatenate(all_labels, axis=0)

print("Shape of all images:", X_all.shape)
print("Shape of all labels:", y_all.shape)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_all)
y_one_hot = to_categorical(y_encoded)

# Reshape data for oversampling
X_reshaped = X_all.reshape(X_all.shape[0], -1)
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_reshaped, y_one_hot)

print("Class distribution after oversampling:", Counter(np.argmax(y_resampled, axis=1)))

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
print(f"Shapes - X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

In [ ]:
# Reshape to (28, 28, 3)
X_train_reshaped = X_train.reshape(X_train.shape[0], 75, 75, 3)
X_test_reshaped = X_test.reshape(X_test.shape[0], 75, 75, 3)

In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
]
# Define input shape
input_shape = (75, 75, 3)
num_classes = 8  # Adjust based on your dataset

# Load Pretrained Models
def build_inception_v1():
    base_model = InceptionV3(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model






# Train and Evaluate Models
def train_and_evaluate(model, name):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test),
                        epochs=5, batch_size=64, callbacks=callbacks)
    test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
    print(f"{name} Test Accuracy: {test_accuracy * 100:.2f}%")
    return history, test_accuracy

# Train models
models = {
    "InceptionV1": build_inception_v1(),
}

histories, accuracies = {}, {}
for name, model in models.items():
    histories[name], accuracies[name] = train_and_evaluate(model, name)

# Plot Comparison
plt.figure(figsize=(10, 5))
plt.bar(accuracies.keys(), accuracies.values(), color=['red', 'blue', 'green', 'purple'])
plt.xlabel("Model")
plt.ylabel("Test Accuracy")
plt.title("Comparison of CNN Models")
plt.show()

In [ ]:
import json
import matplotlib.pyplot as plt
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define callback
callbacks = [EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)]

# Define input shape & number of classes
input_shape = (75, 75, 3)
num_classes = 8  # Adjust based on your dataset

# Build InceptionV3 Model
def build_inception_v3():
    base_model = InceptionV3(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Train & Save Model
def train_and_save(model, model_name="InceptionV3"):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test),
                        epochs=10, batch_size=64, callbacks=callbacks)
    
    test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
    print(f"{model_name} Test Accuracy: {test_accuracy * 100:.2f}%")

    # Save model
    model.save(f"{model_name}.h5")

    # Save training history and accuracy in JSON
    history_dict = history.history
    history_dict["test_accuracy"] = test_accuracy  # Store test accuracy separately

    with open(f"{model_name}_history.json", "w") as f:
        json.dump(history_dict, f)

    return history, test_accuracy

# Train and Save InceptionV3
model = build_inception_v3()
history, accuracy = train_and_save(model, "InceptionV3")

# Save Accuracy Separately
accuracy_data = {"InceptionV3": accuracy}
with open("model_accuracies.json", "w") as f:
    json.dump(accuracy_data, f)

# Plot & Save Accuracy Graph
plt.figure(figsize=(6, 4))
plt.bar(["InceptionV3"], [accuracy * 100], color="green")
plt.xlabel("Model")
plt.ylabel("Test Accuracy (%)")
plt.title("InceptionV3 Accuracy")
plt.savefig("inceptionv3_accuracy.png")  # Save plot as an image file
plt.show()


ALEXNET_______________________

In [ ]:
def load_and_resize_images(image_paths, target_size=(128, 128)):  # Resize correctly for AlexNet
    images = []
    for path in image_paths:
        img = load_img(path, target_size=target_size)  # Resize the image
        img_array = img_to_array(img)  # Convert to array
        images.append(img_array)
    return np.array(images)


In [ ]:
# Define classes
class_folders = ['MEL', 'VASC', 'SCC', 'DF', 'NV', 'BKL', 'BCC', 'AK']
# Load images and labels
all_images, all_labels = [], []

for class_name in class_folders:
    class_folder_path = os.path.join(dataset_path, class_name)
    image_files = os.listdir(class_folder_path)
    class_images = load_and_resize_images([os.path.join(class_folder_path, img) for img in image_files], target_size=(128, 128))
    all_images.append(class_images)
    all_labels.append([class_name] * len(image_files))

# Combine all data
X_all = np.concatenate(all_images, axis=0)
y_all = np.concatenate(all_labels, axis=0)

print("Shape of all images:", X_all.shape)  # Should be (total_images, 227, 227, 3)
print("Shape of all labels:", y_all.shape)



In [ ]:
# Now the images are correctly sized, and we can proceed with splitting
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# No need to reshape manually, as images are already (227, 227, 3)
print(f"Shapes - X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_alexnet(input_shape=(128, 128, 3), num_classes=8):  
    model = Sequential([
        Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape),  # Modify kernel size if needed
        MaxPooling2D((3, 3), strides=(2, 2)),
        
        Conv2D(256, (5, 5), padding='same', activation='relu'),
        MaxPooling2D((3, 3), strides=(2, 2)),
        
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D((3, 3), strides=(2, 2)),
        
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
# Build the model
alexnet_model = build_alexnet()
alexnet_model.summary()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Convert class labels (strings) to numeric values
label_encoder = LabelEncoder()
y_all_numeric = label_encoder.fit_transform(y_all)  # Convert string labels to numbers

# Split Data (Ensure you already have X_train, X_test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all_numeric, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=8)
y_test = to_categorical(y_test, num_classes=8)

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# Define callback
callbacks = [EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)]
# Reshape to (28, 28, 3)
X_train_reshaped = X_train.reshape(X_train.shape[0], 128, 128, 3)
X_test_reshaped = X_test.reshape(X_test.shape[0], 128, 128, 3)# Train & Save Model

def train_and_save(model, model_name="AlexNet"):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test),
                        epochs=10, batch_size=64, callbacks=callbacks)
    
    test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test, verbose=1)
    print(f"{model_name} Test Accuracy: {test_accuracy * 100:.2f}%")

    # Save model
    model.save(f"{model_name}.h5")

    # Save training history and accuracy in JSON
    history_dict = history.history
    history_dict["test_accuracy"] = test_accuracy  # Store test accuracy separately

    with open(f"{model_name}_history.json", "w") as f:
        json.dump(history_dict, f)

    return history, test_accuracy

# Train and Save AlexNet
alexnet_model = build_alexnet()
history, accuracy = train_and_save(alexnet_model, "AlexNet")

# Save Accuracy Separately
accuracy_data = {"AlexNet": accuracy}
with open("model_accuracies.json", "w") as f:
    json.dump(accuracy_data, f)

# Plot & Save Accuracy Graph
plt.figure(figsize=(6, 4))
plt.bar(["AlexNet"], [accuracy * 100], color="blue")
plt.xlabel("Model")
plt.ylabel("Test Accuracy (%)")
plt.title("AlexNet Accuracy")
plt.savefig("alexnet_accuracy.png")  # Save plot as an image file
plt.show()

novality-----CNN deep architecture 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, 
                                     MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Add, Concatenate)

def residual_block(x, filters):
    shortcut = x  # Save input for residual connection
    
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    
    x = Add()([x, shortcut])  # Residual connection
    x = Activation('relu')(x)
    return x

def dense_block(x, filters):
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    
    x = Concatenate()([x, conv2])  # Dense connection
    return x

def build_novel_cnn(input_shape=(75, 75, 3), num_classes=8):
    inputs = Input(shape=input_shape)
    
    # Initial Convolution
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    # Residual & Dense Blocks
    x = residual_block(x, 64)
    x = dense_block(x, 64)
    x = MaxPooling2D((2, 2))(x)
    
    x = residual_block(x, 128)
    x = dense_block(x, 128)
    x = MaxPooling2D((2, 2))(x)
    
    x = residual_block(x, 256)
    x = dense_block(x, 256)
    x = MaxPooling2D((2, 2))(x)
    
    # Global Pooling & Fully Connected Layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, x, name="Novel_CNN")
    return model

# Create the model
novel_cnn = build_novel_cnn()
novel_cnn.summary()
